[Code](https://github.com/ProsusAI/finBERT/blob/master/finbert/finbert.py)

[Theory](https://medium.com/prosus-ai-tech-blog/finbert-financial-sentiment-analysis-with-bert-b277a3607101)

[Evaluate](https://huggingface.co/docs/transformers/v4.39.3/en/tasks/sequence_classification#evaluate) - [Github](https://github.com/huggingface/evaluate/tree/main/metrics/accuracy)

[Rougue Metric](https://huggingface.co/spaces/evaluate-metric/rouge?source=post_page-----fb60abdeba07--------------------------------)

[Choosing right metric](https://huggingface.co/docs/evaluate/en/choosing_a_metric)

# [Check Models memory usage](https://huggingface.co/spaces/Vokturz/can-it-run-llm)

In [ ]:
!pip install accelerate bitsandbytes

In [ ]:
# !accelerate estimate-memory bert-base-cased
!accelerate estimate-memory ProsusAI/finbert

Loading pretrained config for `HuggingFaceM4/idefics-80b-instruct` from `transformers`...
┌──────────────────────────────────────────────────────────────────┐
│  Memory Usage for loading `HuggingFaceM4/idefics-80b-instruct`   │
├───────┬─────────────┬──────────┬─────────────────────────────────┤
│ dtype │Largest Layer│Total Size│       Training using Adam       │
├───────┼─────────────┼──────────┼─────────────────────────────────┤
│float32│   3.02 GB   │297.12 GB │             1.16 TB             │
│float16│   1.51 GB   │148.56 GB │            594.24 GB            │
│  int8 │  772.52 MB  │ 74.28 GB │               N/A               │
│  int4 │  386.26 MB  │ 37.14 GB │               N/A               │
└───────┴─────────────┴──────────┴─────────────────────────────────┘


# Base Model ID

In [ ]:
MODEL_ID = "ProsusAI/finbert"
REPOSITORY_ID = f"{MODEL_ID.split('/')[1]}-News-Headlines-SA-fine-tuned"

# Evaluate model before fine tuning

[Pipelines](https://huggingface.co/docs/transformers/en/pipeline_tutorial)

[link2](https://huggingface.co/docs/transformers/en/main_classes/pipelines)

**[Load Model](https://huggingface.co/ProsusAI/finbert)**

In [ ]:
# from transformers import pipeline

from transformers import AutoTokenizer, AutoModelForSequenceClassification

# from transformers import AutoTokenizer, FalconForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
finbert_model = AutoModelForSequenceClassification.from_pretrained(MODEL_ID)


[Load data](https://huggingface.co/datasets/financial_phrasebank)

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("financial_phrasebank")

**Tokenize**

In [ ]:
inputs = tokenizer(text, return_tensors="pt")

**Prediction**

In [ ]:
with torch.no_grad():
  logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()

In [ ]:
# prompt: code to evaluate transformer model on the dataset and calculate precision, recall, accuracy and F1 score

def tokenize_datasets():
  inputs = tokenizer(texts_to_classify,padding="max_length",truncation=True,max_length=200,return_tensors="pt",)

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Define the evaluation metric
metric = evaluate.load("accuracy")

# Define the evaluation function
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels)

# Evaluate the model
finbert_model.eval()
predictions = finbert_model.predict(tokenized_datasets["train"])
metrics = compute_metrics(predictions)

# Print the results
print(f"Precision: {metrics['precision']}")
print(f"Recall: {metrics['recall']}")
print(f"Accuracy: {metrics['accuracy']}")
print(f"F1 Score: {metrics['f1']}")


In [ ]:
# prompt: code to evaluate transformer model on the dataset and calculate precision, recall, accuracy and F1 score

import numpy as np
# Load the dataset
dataset = load_dataset("glue", "mrpc")

# Split the dataset into training and validation sets
train_dataset = dataset["train"]
validation_dataset = dataset["validation_matched"]

# Preprocess the data
def preprocess_function(examples):
  return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_validation_dataset = validation_dataset.map(preprocess_function, batched=True)

# Define the evaluation metric
metric = evaluate.load("glue", "mrpc")

# Evaluate the model on the validation set
predictions = model.predict(tokenized_validation_dataset)
metric_result = metric.compute(predictions=np.argmax(predictions.predictions, axis=1), references=validation_dataset["label"])

# Print the results
print(f"Precision: {metric_result['precision']}")
print(f"Recall: {metric_result['recall']}")
print(f"Accuracy: {metric_result['accuracy']}")
print(f"F1 Score: {metric_result['f1']}")


In [ ]:
#using device_map requires installing accelerate
#load in 8 bit requires installing bitsandbytes
#!pip install accelerate bitsandbytes

# pipe = pipeline("text-generation", model="tiiuae/falcon-7b", trust_remote_code=True, device_map="auto", model_kwargs={"load_in_8bit": True}) #model_kwargs={"load_in_8bit": True}




In [ ]:
from sklearn.metrics import classification_report

# Inference on Financial Phrasebank dataset

[Financial phrasebank](https://huggingface.co/datasets/financial_phrasebank)

[Link2](https://huggingface.co/datasets/ChanceFocus/en-fpb?row=6)

# Inference on News article headlines

[Prompting](https://pub.towardsai.net/prompt-engineering-best-practices-text-summarization-information-retrieval-bb5abbfcf618)